In [ ]:
# Step 1: Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle

In [ ]:
# Step 2: Load dataset
df = pd.read_csv('../data/processed/dataset.csv')
df.head()

In [ ]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [ ]:
# Step 3: Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# Step 4: Initialize model
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)

# Step 5: Train the model
rf_model.fit(X_train, y_train)

In [ ]:
# Step 6: Make predictions
y_pred = rf_model.predict(X_test)

# Step 7: Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# Step 8: Save model
with open('../models/random_forest.pkl', 'wb') as f:
    pickle.dump(rf_model, f)

print("Random Forest model saved successfully as pickle!")